## **Setting Up Bing Integration (Tool) with Azure AI Agents**

### **📝 Overview**

Grounding with Bing Search enables your Azure AI Agents to incorporate real-time public web data when generating responses. By leveraging Bing Search, your agent can fetch the latest information—such as top news or industry updates—and return relevant, cited content. 

When a user sends a query, the agent first determines if it should invoke Bing Search. If so, it sends a Bing search query (using only your resource key for billing and rate limiting) and retrieves search results. These results are then used to generate a final, human-readable response that includes required citations and links, as specified by Microsoft’s use and display requirements.

> **Important**: Usage of Grounding with Bing Search can incur additional costs. For pricing and legal details, review the [Bing Grounding Terms](https://www.microsoft.com).


### **✅ Prerequisites**

1. **Grounding with Bing Search Resource**
   - Create a Grounding with Bing Search resource using the Azure portal or a code-first approach.
   - Ensure you have **Owner** or **Contributor** permissions to register the resource provider `Microsoft.Bing` if using a code-first approach.

2. **Azure AI Agent Setup**
   - Ensure you have an Azure AI Agent created (via the Agent Playground or quickstart).

3. **Permission/Role Assignment**
   - **Access to Bing Search**: Users do not have access to raw search content; instead, they receive model-generated responses that include citations and links.
   - **RBAC for Foundry Project**: End users must have the `AI Developer` role assigned.

4. **Supported Models**
   - Grounding with Bing Search currently works with specific Azure OpenAI models (e.g., `gpt-3.5-turbo-0125`, `gpt-4-0125-preview`, etc.).

5. **Create a New Connection**
   - Provide the required key-value pairs for the connection:
   - **`resource-key`**: Your Bing resource key.
   - **`endpoint`**: Your Bing search endpoint (typically provided in your resource details).

6. **Add a Knowledge Source**
   - Click to add a knowledge source and select **Grounding with Bing Search**.
   - If you don’t see this option:
      - Verify that your resource has been created in the same resource group as your AI Agent.
      - Ensure you have the necessary permissions.


In [12]:
import os

from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Define the target directory
target_directory = os.getcwd()  # Get the current working directory

# Move one directory back
parent_directory = os.path.dirname(target_directory)

# Check if the parent directory exists
if os.path.exists(parent_directory):
    # Change the current working directory to the parent directory
    os.chdir(parent_directory)
    print(f"Directory changed to {os.getcwd()}")
else:
    print(f"Parent directory {parent_directory} does not exist.")

Directory changed to c:\Users\pablosal\Desktop


In [13]:
import importlib.metadata as md

# Versions - we are currently 1.0.0b9 of azure-ai-projects
print("semantic-kernel version:", md.version("semantic-kernel"))
# if you want to Upgrade the SDKs, uncomment the line below but code might break
# %pip install -U semantic-kernel azure-ai-projects azure-identity

semantic-kernel version: 1.28.1


In [3]:
import os
import re
import time
import logging
import json
from datetime import datetime as pydatetime
from typing import Any, List, Dict, Optional
from dotenv import load_dotenv
import asyncio
from datetime import timedelta

# Azure AI Projects
from azure.identity.aio import DefaultAzureCredential
from azure.core.exceptions import HttpResponseError

# semantic kernel
from semantic_kernel.contents import AuthorRole
from semantic_kernel.agents.azure_ai.azure_ai_agent import (
    AzureAIAgent,
    AzureAIAgentSettings,
)
from semantic_kernel.agents.open_ai.run_polling_options import RunPollingOptions

# Load environment variables from .env file
load_dotenv()

# configure logging
from utils.ml_logging import get_logger

logger = get_logger()

In [37]:
import os
import json
import logging
from typing import Set, Tuple, Optional
from azure.core.exceptions import ServiceRequestError
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient


def process_citations(text_msg) -> str:
    """
    Given a text_message with .text.value and .text.annotations, append a
    **Citations** section with unique URL citations.
    """
    base = text_msg.text.value
    seen: Set[Tuple[str, str]] = set()

    for annot in getattr(text_msg.text, "annotations", []):
        uc = getattr(annot, "url_citation", None)
        if uc and uc.url:
            seen.add((annot.text, uc.url))

    if seen:
        base += "\n\n**Citations:**\n"
        for quote, url in seen:
            base += f"- **Quote**: {quote}  \n"
            base += f"  **URL**: [{url}]({url})\n"

    return base


def run_agent(
    project_client: AIProjectClient, agent_id: str, user_input: str
) -> Tuple[str, str]:
    """
    • Posts `user_input` to a new thread on `agent_id`.
    • Blocks until the run completes.
    • Gathers only the *real* assistant replies, enriches with citations.
    Returns (conversation_text, thread_id).
    """
    # 1) create thread & send question
    thread = project_client.agents.create_thread()
    project_client.agents.create_message(
        thread_id=thread.id, role="user", content=user_input
    )

    # 2) run & wait
    project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent_id)

    # 3) collect & enrich only true assistant messages
    convo = f"👤 User: {user_input}\n"
    pager = project_client.agents.list_messages(thread_id=thread.id)
    messages = pager.data if hasattr(pager, "data") else list(pager)

    for msg in messages:
        # only look at assistant turns
        if msg.role.lower() != "assistant":
            continue

        for text_msg in getattr(msg, "text_messages", []):
            content = text_msg.text.value.strip()

            # skip exact echoes of the prompt
            if content == user_input.strip():
                continue

            enriched = process_citations(text_msg)
            convo += f"\n🤖 Assistant: {enriched}\n"

    return convo, thread.id

### **Create Client and Load Azure AI Foundry**

Here, we initialize the Azure AI client using DefaultAzureCredential. This allows us to authenticate and connect to the Azure AI service.

In [16]:
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ["AZURE_AI_FOUNDRY_CONNECTION_STRING"],
)

In [19]:
from azure.ai.projects.models import (
    BingGroundingTool,
    ToolSet,
)


def get_connection_id(client: AIProjectClient, env_var: str) -> Optional[str]:
    """
    Retrieves the connection object using a connection name stored in an environment variable.

    Args:
        client: The Azure AI Project client.
        env_var (str): The environment variable holding the connection name.

    Returns:
        Connection object if found, otherwise raises an error.
    """
    connection_name = os.getenv(env_var)
    if not connection_name:
        logger.error(f"Missing environment variable: '{env_var}'")
        raise ValueError(f"Environment variable '{env_var}' is required.")

    try:
        connection = client.connections.get(connection_name=connection_name)
        logger.info(f"Retrieved Connection ID for {env_var}: {connection.id}")
        return connection
    except Exception as e:
        logger.error(f"Failed to retrieve connection for {env_var}: {e}")
        raise


# Initialize Azure AI Agent settings
researcheragent_settings = AzureAIAgentSettings.create()

# Create a ToolSet to manage tools
toolset = ToolSet()

try:
    # Retrieve and add the Bing Grounding Tool
    bing_connection = get_connection_id(project_client, "TOOL_CONNECTION_NAME_BING")
    toolset.add(BingGroundingTool(connection_id=bing_connection.id))
    logger.info("Bing Grounding Tool added successfully.")

    logger.info("Successfully created ToolSet with Bing and File Search tools.")
except Exception as e:
    logger.error(f"Failed to create ToolSet: {e}")
    raise

The create method is deprecated. Use the __new__ method instead.


2025-05-28 20:50:00,407 - micro - MainProcess - INFO     Retrieved Connection ID for TOOL_CONNECTION_NAME_BING: /subscriptions/47f1c914-e299-4953-a99d-3e34644cfe1c/resourceGroups/rg-zhuoqunliai/providers/Microsoft.MachineLearningServices/workspaces/zhuoqunli-1959/connections/agentsbinggrounding (2871371040.py:get_connection_id:24)
2025-05-28 20:50:00,408 - micro - MainProcess - INFO     Bing Grounding Tool added successfully. (2871371040.py:<module>:40)
2025-05-28 20:50:00,410 - micro - MainProcess - INFO     Successfully created ToolSet with Bing and File Search tools. (2871371040.py:<module>:42)


In [21]:
researcheragent_settings_definition = project_client.agents.create_agent(
    model=researcheragent_settings.model_deployment_name,
    name="BingDataRetrievalAgent",
    description=(
        "An AI agent specialized in retrieving and analyzing real-time data from the web using Bing. "
        "This includes retrieving up-to-date information, news, research articles, and other publicly available web content. "
        "The agent is designed to assist in research and decision-making by providing accurate, relevant, and actionable insights. "
        "If no relevant data is found, the agent must clearly indicate this and provide suggestions for refining the query."
    ),
    instructions=(
        "### Role & Objective\n"
        "You are a research-focused AI assistant responsible for retrieving and analyzing real-time data exclusively from the web using Bing. "
        "Your goal is to provide precise, well-referenced, and relevant responses to support research, decision-making, and analysis efforts.\n\n"
        "### Data Retrieval & Prioritization\n"
        "1. **Real-Time Web Data (Bing):** \n"
        "   - Use Bing to retrieve real-time data, including news, research articles, and other publicly available web content.\n"
        "   - Focus on extracting key insights, summaries, and actionable information from the retrieved web pages.\n"
        "   - Example: 'Retrieve the latest news on advancements in AI technology,' or 'Find research articles on the impact of climate change on agriculture.'\n\n"
        "2. **Source Evaluation:** \n"
        "   - Prioritize credible and authoritative sources (e.g., academic journals, government websites, reputable news outlets).\n"
        "   - Avoid using unreliable or unverified sources.\n\n"
        "3. **Integrated Queries:** \n"
        "   - If the query requires multiple pieces of information, retrieve and integrate the results for a comprehensive response.\n"
        "   - Ensure clarity in presenting combined insights.\n\n"
        "4. **Fallback Behavior:** \n"
        "   - If no relevant data is found on the web, respond with:\n"
        "     - A clear statement that no relevant data was found.\n"
        "     - Suggestions for refining the query or exploring alternative approaches.\n"
        "     - Example: 'No relevant data was found for the requested query on the web. Consider refining your query or specifying additional details.'\n\n"
        "### Response Quality\n"
        "1. **Accuracy & Relevance:** Always prioritize retrieving the most current and applicable data from credible sources.\n"
        "2. **Clarity & Transparency:** Clearly indicate the data source(s) used and any limitations in the available information.\n"
        "3. **Fallback Handling:** If no relevant data is found, provide a professional and helpful fallback response as outlined above.\n"
        "4. **Professionalism:** Present findings in a structured and concise manner to facilitate decision-making.\n"
        "5. **Source Context:** Ensure that extracted insights are presented with sufficient context to maintain their relevance and accuracy.\n"
    ),
    toolset=toolset,
    headers={"x-ms-enable-preview": "true"},
    temperature=0.7,
    top_p=1,
    metadata={
        "use_case": "Real-Time Web Data Retrieval for Research and Decision-Making",
        "data_source": "Bing (web search)",
    },
)

# Print the agent's run ID (agent ID)
print(f"BingDataRetrievalAgent Run ID: {researcheragent_settings_definition.id}")

BingDataRetrievalAgent Run ID: asst_CmPBsqtQXXtDsaiQPsN0gLyH


In [ ]:
user_query = """Please provide a summary of the top products currently available on the market for glucose monitoring,
    specifically focusing on studies comparing calibration-free interstitial CGM sensors with capillary blood glucose monitoring by glucometer.
    Highlight key findings, accuracy, advantages, and any notable limitations or clinical implications from recent research."""
research_result, threadID = run_agent(
    project_client, researcheragent_settings_definition.id, user_query
)

In [39]:
print(f"Conversation Text:\n{research_result}")

Conversation Text:
👤 User: Please provide a summary of the top products currently available on the market for glucose monitoring, specifically focusing on studies comparing calibration-free interstitial CGM sensors with capillary blood glucose monitoring by glucometer. Highlight key findings, accuracy, advantages, and any notable limitations or clinical implications from recent research.

🤖 Assistant: Recent studies comparing calibration-free continuous glucose monitoring (CGM) sensors with traditional capillary blood glucose monitoring by glucometer have highlighted several key findings:

1. **Accuracy**: Calibration-free CGMs, such as the Freestyle Libre and Dexcom G6, generally show a mean absolute relative difference (MARD) of around 10% compared to glucometers. This indicates a reliable performance, although there is a natural lag in CGMs due to the time it takes for glucose to equilibrate in interstitial fluid compared to capillary blood【3:1†source】【3:2†source】.

2. **Advantages*